In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne
import pywt
from torch.utils.data import DataLoader, Dataset , Subset
from torch import nn
import torch
import pickle
import os
from datetime import datetime
import sqlite3
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import TensorBoardLogger

from torchmetrics.functional.classification.accuracy import accuracy


In [34]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import statsmodels.api as sm
data = sm.datasets.co2.load_pandas()
#converto to df
df = data.data

In [35]:
df

In [24]:


# Wczytanie danych
data = sm.datasets.airline.load_pandas().data['AirPassengers'].astype(np.float32)
data = sm.datasets.co2.load_pandas().data['co2'].astype(np.float32)
data_normalized = (data - data.min()) / (data.max() - data.min())  # Normalizacja

# Przygotowanie sekwencji
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq, train_label))
    return inout_seq

train_window = 12
train_inout_seq = create_inout_sequences(data_normalized, train_window)

X = torch.FloatTensor([s[0] for s in train_inout_seq])
y = torch.FloatTensor([s[1] for s in train_inout_seq])

train_data = TensorDataset(X, y)
train_loader = DataLoader(train_data, batch_size=1, shuffle=False)

# Definicja modelu LSTM
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super(LSTM, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

# Inicjalizacja modelu
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Trenowanie modelu
epochs = 150
for i in range(epochs):
    for seq, labels in train_loader:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))
        y_pred = model(seq)
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
    if i % 25 == 0:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {epochs-1:3} loss: {single_loss.item():10.10f}')


![title](img/lstm.png)

![title](img/m2m.jpg)


In [13]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from pandas_datareader import data as pdr
import datetime
